In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import random

Using TensorFlow backend.


In [3]:
# Get images
image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype='float32')

In [4]:
image.shape

(400, 400, 3)

Lab color space is a 3-axis color system with dimension `L` for lightness and `a` green–red(-128, 128) and `b` blue–yellow(-128, 128) for the color dimensions

In [14]:
X = rgb2lab(1.0 / 255 * image)[:,:,0] # Get lightness only
Y = rgb2lab(1.0 / 255 * image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [28]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))  # (400, 400, 1)
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))  # (200, 200, 8)
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)) # (100, 100, 16)
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)) # ( 50,  50, 32)
model.add(UpSampling2D((2, 2)))    # (100, 100, 16)
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))    # ( 200, 200, 32)
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))    # ( 400, 400, 16)
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, None, None, 8)     80        
_________________________________________________________________
conv2d_35 (Conv2D)           (None, None, None, 8)     584       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, None, None, 16)    1168      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, None, None, 16)    2320      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, None, None, 32)    4640      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, None, None, 32)    9248      
__________

In [30]:
model.compile(optimizer='rmsprop',
              loss = 'mse')

In [31]:
model.fit(X,
          Y,
          batch_size=1,
          epochs=1000,
          verbose=0)

In [32]:
print(model.evaluate(X, Y, batch_size=1))

1/1 [==============================] - 0s
0.000343502149917


In [36]:
output = model.predict(X)
output *=128

# Output colorizations
cavas = np.zeros((400, 400, 3))
cavas[:, :, 0] = X[0][:, :, 0]
cavas[:, :, 1:] = output[0]
imsave("img_woman.jpg", lab2rgb(cavas))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cavas)))

C:\Users\wuht\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\wuht\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
